# TfNSW Data Exploration

## Objective

Explore TfNSW Sydney Trains data in Python, similar to the analysis in R.

## Setup

In [128]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import altair as alt
#import plotly.express as px
#import plotly.io as pio
#import plotly.graph_objects as go

#pio.templates.default = 'plotly_dark'

## Import data

In [2]:
routes = pd.read_csv("data/routes.csv") \
    .query('agency_id == "SydneyTrains"') \
    .drop(['agency_id', 'route_type', 'route_url', 'route_text_color'], axis = 1) \
    .dropna()

routes.head(5)

,route_id,route_short_name,route_long_name,route_desc,route_color
0,APS_1a,T8,City Circle to Macarthur via Airport,T8 Airport & South Line,00954C
1,APS_1b,T8,City Circle to Leppington via Airport,T8 Airport & South Line,00954C
2,APS_1c,T8,City Circle to Macarthur via Sydenham,T8 Airport & South Line,00954C
3,APS_1d,T8,City Circle to Macarthur via Sydenham,T8 Airport & South Line,00954C
4,APS_1e,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,00954C


In [3]:
stops = pd.read_csv("data/stops.csv") \
    .drop(['stop_timezone', 'wheelchair_boarding', 'stop_desc', 'zone_id', 'stop_url'], axis = 1)
stops.head(5)

,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station
0,26401,26401,Albury Station,-36.084068,146.924691,1,NaN
1,264086,264086,Albury Station Platform 1,-36.083941,146.924558,0,26401
2,233610,233610,Aberdeen Station,-32.167104,150.892052,1,NaN
3,233621,233621,Aberdeen Station Platform 1,-32.166969,150.892016,0,233610
4,222020,222020,Allawah Station,-33.969584,151.114330,1,NaN


In [4]:
trips = pd.read_csv("data/trips.csv") \
    .drop(['service_id', 'trip_short_name', 'block_id', 'wheelchair_accessible'], axis = 1)
trips.head(10)

,route_id,trip_id,trip_headsign,direction_id,shape_id
0,RTTA_REV,1--A.1259.126.128.B.8.59842229,Empty Train,0,RTTA_REV
1,RTTA_REV,1--A.1259.126.2.B.8.59842229,Empty Train,0,RTTA_REV
2,BNK_2a,1--A.1260.127.124.B.8.59851802,City Circle via Museum,1,BNK_2a
3,BNK_2a,1--A.1495.102.124.B.8.59416274,City Circle via Museum,1,BNK_2a
4,RTTA_REV,1--A.1495.102.130.B.8.59416276,Empty Train,0,RTTA_REV
5,RTTA_REV,1--A.1620.103.134.B.8.59625698,Empty Train,0,RTTA_REV
6,BNK_2a,1--A.483.104.56.B.8.59918823,City Circle via Museum,1,BNK_2a
7,BNK_2a,1--A.483.104.64.B.8.59918823,City Circle via Museum,1,BNK_2a
8,BNK_2d,1--B.1259.126.128.B.8.59842228,City Circle via Town Hall,1,BNK_2d
9,BNK_2d,1--B.1259.126.2.B.8.59842228,City Circle via Town Hall,1,BNK_2d


In [8]:
stop_times = pd.read_csv("data/stop_times.csv", low_memory = False) \
    .drop(['arrival_time', 'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type', "shape_dist_traveled"], axis = 1)

stop_times.head(5)

,trip_id,stop_id,stop_sequence
0,1--A.1259.126.128.B.8.59842229,2142324,8
1,1--A.1259.126.128.B.8.59842229,2160262,13
2,1--A.1259.126.128.B.8.59842229,2161252,14
3,1--A.1259.126.128.B.8.59842229,2161262,17
4,1--A.1259.126.128.B.8.59842229,2165202,19


In [9]:
stop_seq = pd.merge(stop_times, stops, on = 'stop_id')
stop_seq = pd.merge(stop_seq, trips, on = 'trip_id') \
    .drop(['stop_lat', 'stop_lon', 'stop_code'], axis = 1)

stop_seq.head(5)

,trip_id,stop_id,stop_sequence,stop_name,location_type,parent_station,route_id,trip_headsign,direction_id,shape_id
0,1--A.1259.126.128.B.8.59842229,2142324,8,Granville Station Platform 4,0,214240,RTTA_REV,Empty Train,0,RTTA_REV
1,1--A.1259.126.128.B.8.59842229,2160262,13,Merrylands Station Platform 2,0,216010,RTTA_REV,Empty Train,0,RTTA_REV
2,1--A.1259.126.128.B.8.59842229,2161252,14,Guildford Station Platform 2,0,216110,RTTA_REV,Empty Train,0,RTTA_REV
3,1--A.1259.126.128.B.8.59842229,2161262,17,Yennora Station Platform 2,0,216120,RTTA_REV,Empty Train,0,RTTA_REV
4,1--A.1259.126.128.B.8.59842229,2165202,19,Fairfield Station Platform 2,0,216510,RTTA_REV,Empty Train,0,RTTA_REV


In [8]:
# shapes = pd.read_csv("data/shapes.csv")
# shapes.head(5)

## Distinct trips

In [10]:
trips_per_route = pd.merge(trips, routes, on = 'route_id') \
    .groupby(by = ['route_long_name', 'route_color']) \
    .count() \
    .filter(items=['trip_id']) \
    .sort_values('trip_id', ascending = False)

In [10]:
len(trips_per_route)

49

In [11]:
trips_per_route.head(5)

,,trip_id
route_long_name,route_color,
City to Berowra via Gordon,F99D1C,3474
North Shore to Hornsby and Berowra via City,D11F2F,2770
Berowra and Hornsby to City via Gordon,F99D1C,2381
City Circle to Macarthur via Airport,00954C,2086
Macarthur to City Circle via Airport,00954C,1934


In [94]:
route_colours = pd.DataFrame(data = np.array(routes.route_color.drop_duplicates().map(lambda x: "#" + x)),
                             columns = ['rgb'],
                             index = routes.route_color.drop_duplicates())
route_colours

,rgb
route_color,
00954C,#00954C
F37021,#F37021
C4258F,#C4258F
005AA3,#005AA3
0098CD,#0098CD
F99D1C,#F99D1C
D11F2F,#D11F2F
6F818E,#6F818E


In [122]:
alt.Chart(trips_per_route.reset_index()) \
    .mark_bar(
        tooltip = True) \
    .encode(
        x = alt.X(
            'trip_id', 
            title = 'Number of trips'),
        y = alt.Y(
            'route_long_name', 
            sort = '-x', 
            title = None),
        color = alt.Color(
            'route_color', 
            scale = alt.Scale(
                domain = route_colours.index.to_list(), 
                range = route_colours['rgb'].to_list()),
            legend = None)) \
    .properties(
        width = 800,
        height = 600,
        title = 'Trips per route') \
    .configure_title(
        fontSize = 20,
        color = '#666666',
        dy = -20,
        anchor = 'start')

alt.Chart(...)

## Stops per trip

In [123]:
def stops_by_trip_id (id):
    
    x = stop_seq[stop_seq.trip_id == id].drop_duplicates()
    return pd.merge(x, stops, on = 'stop_id') \
        .filter(['stop_name_x'], axis=1)


def stops_by_route_id (id):
    
    x = trips[trips.route_id == id] \
        .head(1) \
        .trip_id \
        .item()
    return stops_by_trip_id(x)


In [124]:
stops_by_route_id('CCN_1a')

,stop_name_x
0,Redfern Station Platform 6
1,Macdonaldtown Station Platform 2
2,Newtown Station Platform 2
3,Stanmore Station Platform 3
4,Petersham Station Platform 2
5,Lewisham Station Platform 2
6,Summer Hill Station Platform 3
7,Ashfield Station Platform 5
8,Croydon Station Platform 5
9,Burwood Station Platform 6


In [125]:
stops_by_trip_id('N135.1495.102.44.V.8.59426689')

,stop_name_x
0,Meadowbank Station Platform 2
1,West Ryde Station Platform 3
2,Cheltenham Station Platform 2
3,Beecroft Station Platform 2
4,Pennant Hills Station Platform 2
5,Thornleigh Station Platform 3
6,Normanhurst Station Platform 2
7,Asquith Station Platform 2
8,Mount Colah Station Platform 2
9,Mount Kuring-gai Station Platform 2


In [126]:
trips[trips.route_id == 'CCN_1a'].head(1).trip_id.item()

'202B.1260.127.16.H.4.60059035'

In [127]:
stop_seq.tail(5)

,trip_id,stop_id,stop_sequence,stop_name,location_type,parent_station,route_id,trip_headsign,direction_id,shape_id
957332,SP43.483.104.16.P.2.59916941,266348,1,Junee Station Platform 3,0,26631,CTY_S1f,Griffith,0,CTY_S1f
957333,SP43.483.104.16.P.2.59916941,268058,10,Griffith Station Platform 1,0,26801,CTY_S1f,Griffith,0,CTY_S1f
957334,SP43.483.104.16.P.2.59916941,27054,8,Leeton Station Platform 1,0,27051,CTY_S1f,Griffith,0,CTY_S1f
957335,SP43.483.104.16.P.2.59916941,27003,7,Narrandera Station Platform 1,0,27001,CTY_S1f,Griffith,0,CTY_S1f
957336,SP43.483.104.16.P.2.59916941,270122,6,Coolamon Station Platform 1,0,27011,CTY_S1f,Griffith,0,CTY_S1f
